In [1]:
from PIL import Image
import itertools
import numpy as np
import pandas as pd
import warnings
import nltk
import math
import time
import re
import os
import pickle
#Deep-Learning Library
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import glob

Using TensorFlow backend.


In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = "/content/drive/My Drive/Major Project B.Tech Sem 8/Breast Cancer Dataset/"
savingpath="/content/drive/My Drive/Major Project B.Tech Sem 8/"

In [0]:
train_data_dir=path+"Train"
test_data_dir=path+"Test"

#Training paths
data_cnn_Xception_train=savingpath+"Xception/train/data_cnn_Xception_train.npy"
data_filenames_Xception_train=savingpath+"Xception/train/data_filenames_Xception_train.npy"
data_cancerclass_Xception_train=savingpath+"Xception/train/data_cancerclass_Xception_train.npy"
data_cancertype_Xception_train=savingpath+"Xception/train/data_cancertype_Xception_train.npy"
data_mag_Xception_train=savingpath+"Xception/train/data_mag_Xception_train.npy"

#Testing paths
data_cnn_Xception_test=savingpath+"Xception/test/data_cnn_Xception_test.npy"
data_filenames_Xception_test=savingpath+"Xception/test/data_filenames_Xception_test.npy"
data_cancerclass_Xception_test=savingpath+"Xception/test/data_cancerclass_Xception_test.npy"
data_cancertype_Xception_test=savingpath+"Xception/test/data_cancertype_Xception_test.npy"
data_mag_Xception_test=savingpath+"Xception/test/data_mag_Xception_test.npy"

In [6]:
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
import numpy as np

#model = Xception(weights='imagenet', include_top=True)

base_model = Xception(weights='imagenet')
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
image_size = (299, 299)

Instructions for updating:
Colocations handled automatically by placer.
91889664/91884032 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  if __name__ == '__main__':


In [7]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 224, 224
#top_model_weights_path = 'bottleneck_fc_model.h5'
#epochs = 50
#batch_size = 1
print("Started")

def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return "NO" 
    else: 
        return "YES" 

def save_bottlebeck_features_train(train_data=train_data_dir,nb_train_samples=5000,batch_size=1):
    
    #Function to compute VGG-16 CNN for image feature extraction.
        
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    features=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the Xception network
    #model = applications.Xception(include_top=False, weights='imagenet')
    #print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    a=1
    for i in generator.filenames:
        #print(i)
        #print(type)
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        k=0
        for types in cancersubtypes:          
          flagforcancersubtypes=check(i,types)         
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(flagforcancersubtypes=="YES"):
            cancertype.append(codeforcancersubtypes[k])
          k=k+1
         
        # Array for Magnification
        magnificationsformat=["-40-","-100-","-200-","-400-"]
        codeformagnifications=[40,100,200,400]
        j=0
        for types in magnificationsformat:
          
          flagformagnifications=check(i,types)         
          if(flagformagnifications=="YES"):
            mag.append(codeformagnifications[j])
          j=j+1
 
        image_path=train_data_dir+"/"+i
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
    
    print(features)
        
    print(cancerclass)    
    print(cancertype)
    print(mag)
    print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    
    features=np.array(features)
    print(features.shape)
     
    
    print("reshape the trained vector")
    #features = features.reshape((5000,25088))
    print("Trainning done")
    np.save(open(data_cnn_Xception_train, 'wb'), features)
    np.save(open(data_filenames_Xception_train, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_Xception_train, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_Xception_train, 'wb'), np.array(cancertype))
    np.save(open(data_mag_Xception_train, 'wb'), np.array(mag))
    print("npy file saved")

save_bottlebeck_features_train()

print(time.clock() - start_time, "seconds")

Started
Initialized Required data and datagen done
Found 5000 images belonging to 1 classes.
Genrator work finish
[array([0.29910567, 0.2002473 , 0.        , ..., 0.02143418, 0.3116537 ,
       0.        ], dtype=float32), array([0.        , 0.40094796, 0.00247107, ..., 0.3563264 , 0.0300036 ,
       0.05718253], dtype=float32), array([0.04217038, 0.03189724, 0.02822959, ..., 0.02380877, 0.15055768,
       0.01652785], dtype=float32), array([0.10098027, 0.09463661, 0.02931699, ..., 0.00127656, 0.        ,
       0.1163956 ], dtype=float32), array([0.04133794, 0.18327507, 0.01766576, ..., 0.        , 0.00254005,
       0.00513363], dtype=float32), array([0.00052261, 0.17465666, 0.02776408, ..., 0.04415849, 0.10473002,
       0.03489869], dtype=float32), array([8.9543382e-06, 9.1766119e-02, 6.5114103e-02, ..., 1.0473282e-01,
       1.5310139e-02, 4.3240181e-01], dtype=float32), array([4.2188562e-02, 0.0000000e+00, 5.1520303e-02, ..., 3.8045935e-02,
       3.2575936e-05, 6.8313396e-03], d

In [10]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model.h5'
epochs = 50
batch_size = 1
print("Started")

def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return "NO" 
    else: 
        return "YES" 

def save_bottlebeck_features_test(train_data=test_data_dir,nb_train_samples=2900,batch_size=1):
    
    #Function to compute VGG-16 CNN for image feature extraction.
    
    features=[]
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the Xception network
    #model = applications.Xception(include_top=False, weights='imagenet')
    #print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    for i in generator.filenames:
        
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        k=0
        for types in cancersubtypes:          
          flagforcancersubtypes=check(i,types)         
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(flagforcancersubtypes=="YES"):
            cancertype.append(codeforcancersubtypes[k])
          k=k+1
         
        # Array for Magnification
        magnificationsformat=["-40-","-100-","-200-","-400-"]
        codeformagnifications=[40,100,200,400]
        j=0
        for types in magnificationsformat:
          
          flagformagnifications=check(i,types)         
          if(flagformagnifications=="YES"):
            mag.append(codeformagnifications[j])
          j=j+1
          
        image_path=test_data_dir+"/"+i
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
    
    features=np.array(features)
    print(features.shape)
    
    print(cancerclass)    
    print(cancertype)
    print(mag)
    print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    
    print("Trainning done")
    np.save(open(data_cnn_Xception_test, 'wb'), features)
    np.save(open(data_filenames_Xception_test, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_Xception_test, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_Xception_test, 'wb'), np.array(cancertype))
    np.save(open(data_mag_Xception_test, 'wb'), np.array(mag))
    print("npy file saved")
    

save_bottlebeck_features_test()

print(time.clock() - start_time, "seconds , Testing data also saved")

Started
Initialized Required data and datagen done
Found 2900 images belonging to 1 classes.
Genrator work finish
(2900, 2048)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,